In [22]:
import cv2
import os
import numpy as np
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.image import save_img


In [23]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [24]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [25]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [26]:
x = Dense(1024, activation='relu')(x)

In [27]:
predictions = Dense(4, activation='softmax')(x)

In [28]:
predictions

<tf.Tensor 'dense_6/Softmax:0' shape=(None, 4) dtype=float32>

In [29]:
model = Model(inputs=base_model.input, outputs=predictions)

In [30]:
for layer in base_model.layers:
    layer.trainable = False

In [31]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [32]:
model.summary()  #model view 


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, None, None, 3 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, None, None, 3 0           batch_normalization_189[0][0]    
____________________________________________________________________________________________

In [33]:
#path='C:/Users/DesildaToska/Desktop/tesifase2/K-Fold/KFold_1/train/'
path='C:/Users/DESILDAToska/Desktop/tesifase2/K-Fold/KFold_2/train/'

In [34]:
class_values ={'0_45': 1, '45_60': 2, '60_80' : 3, '80_100' : 4}
count = 0
X = None
Y = None
for dir in os.listdir(path):
    lista = os.listdir(path+dir)
    for file in lista:
        count+=1
        if count % 100 == 0:
            print(count)
        img = load_img(path+dir+'/'+file)  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        if X is None:
            X = x.reshape((1,) + x.shape)
        else:
            X = np.append(X,x.reshape((1,) + x.shape), axis=0)
        if Y is None:
            Y = class_values[dir]
        else:
            Y = np.append(Y, class_values[dir])
        #if count>800:
          #  break

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


In [35]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
label = [1,2,3,4]
enc.fit(np.array(label).reshape(-1, 1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [36]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [37]:
Y_enc = enc.transform(Y.reshape(-1, 1)).toarray()

In [38]:
# Split dataset tra trainign e test
X_train, X_test, y_train, y_test = train_test_split(X, Y_enc, test_size=0.3, random_state = 40)

In [39]:
# Effettua il training
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=32,callbacks=[EarlyStopping(monitor='acc', patience=5, min_delta=0.0001)])

Train on 1856 samples, validate on 796 samples
Epoch 1/30
1856/1856 [==============================] - 219s 118ms/step - loss: 1.5377 - val_loss: 58.9557
Epoch 2/30


C:\Users\DESILDAToska\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `acc` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1856/1856 [==============================] - 235s 127ms/step - loss: 0.6549 - val_loss: 98.9379
Epoch 3/30
1856/1856 [==============================] - 236s 127ms/step - loss: 0.5522 - val_loss: 82.8844
Epoch 4/30
1856/1856 [==============================] - 237s 128ms/step - loss: 0.4315 - val_loss: 111.9599
Epoch 5/30
1856/1856 [==============================] - 250s 135ms/step - loss: 0.3771 - val_loss: 95.8391
Epoch 6/30
1856/1856 [==============================] - 268s 145ms/step - loss: 0.3405 - val_loss: 82.2602
Epoch 7/30
1856/1856 [==============================] - 265s 143ms/step - loss: 0.3187 - val_loss: 110.7406
Epoch 8/30
1856/1856 [==============================] - 264s 142ms/step - loss: 0.3085 - val_loss: 101.5734
Epoch 9/30
1856/1856 [==============================] - 257s 138ms/step - loss: 0.2184 - val_loss: 130.6021
Epoch 10/30
1856/1856 [==============================] - 258s 139ms/step - loss: 0.2770 - val_loss: 103.5273
Epoch 11/30
1856/1856 [===================